In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/'My Drive'/Diploma

/content/gdrive/My Drive/Diploma


In [3]:
!pip install segmentation-models-pytorch
from zipfile import ZipFile
import useful
import numpy as np
import torch
import segmentation_models_pytorch as smp
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.utils.data import Dataset, DataLoader
from data import DepthDataset, RandomHorizontalFlip, ToTensorTest, ToTensorTrain, RandomRotate
from torchvision import transforms
from PIL import ImageFilter
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

     |████████████████████████████████| 71kB 5.4MB/s 
     |████████████████████████████████| 245kB 15.4MB/s 
     |████████████████████████████████| 61kB 10.3MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp37-none-any.whl size=60963 sha256=96547be4534c07c1116bea58cc74393cbe0e7f7edabfa8350f42ab7bfe6002c8
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp37-none-any.whl size=12420 sha256=f1301de428746c4aa3657a77c899749537ea6992f112baa7897e1907a9e1e075
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built pretrainedmodels efficientnet-pytorch


In [4]:
def berHuLoss(pred, target):
    huber_c = torch.max(pred - target)
    huber_c = 0.2 * huber_c
    valid_mask = (target > 0).detach()
    diff = target - pred
    diff = diff[valid_mask]
    diff = diff.abs()
    huber_mask = (diff > huber_c).detach()
    diff2 = diff[huber_mask]
    diff2 = diff2 ** 2
    loss = torch.cat((diff, diff2)).mean()
    return loss

def edges_loss(pred, target, alpha=1):
    def gradient(x):
        h_x = x.size()[-2]
        w_x = x.size()[-1]
        left = x
        right = F.pad(x, [0, 1, 0, 0])[:, :, :, 1:]
        top = x
        bottom = F.pad(x, [0, 0, 0, 1])[:, :, 1:, :]
        dx, dy = right - left, bottom - top 
        dx[:, :, :, -1] = 0
        dy[:, :, -1, :] = 0
        return dx, dy
    gen_dx, gen_dy = gradient(pred)
    gt_dx, gt_dy = gradient(target)
    grad_diff_x = torch.abs(gt_dx - gen_dx)
    grad_diff_y = torch.abs(gt_dy - gen_dy)

    return torch.mean(grad_diff_x ** alpha + grad_diff_y ** alpha)


def gaussian(window_size, sigma):
    gauss = torch.Tensor([math.exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

def create_window(window_size, channel=1):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
    return window

def ssim(img1, img2, val_range, window_size=11, window=None, size_average=True, full=False):
    L = val_range

    padd = 0
    (_, channel, height, width) = img1.size()
    if window is None:
        real_size = min(window_size, height, width)
        window = create_window(real_size, channel=channel).to(img1.device)

    mu1 = F.conv2d(img1, window, padding=padd, groups=channel)
    mu2 = F.conv2d(img2, window, padding=padd, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=padd, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=padd, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=padd, groups=channel) - mu1_mu2

    C1 = (0.01 * L) ** 2
    C2 = (0.03 * L) ** 2

    v1 = 2.0 * sigma12 + C2
    v2 = sigma1_sq + sigma2_sq + C2
    cs = torch.mean(v1 / v2)  # contrast sensitivity

    ssim_map = ((2 * mu1_mu2 + C1) * v1) / ((mu1_sq + mu2_sq + C1) * v2)

    if size_average:
        ret = ssim_map.mean()
    else:
        ret = ssim_map.mean(1).mean(1).mean(1)

    if full:
        return ret, cs

    return ret

In [5]:
class RandomChannelSwap(object):
    def __init__(self, probability):
        from itertools import permutations
        self.probability = probability
        self.indices = list(permutations(range(3), 3))

    def __call__(self, sample):
        image, depth = sample['image'], sample['depth']
        if not _is_pil_image(image): raise TypeError('img should be PIL Image. Got {}'.format(type(image)))
        if not _is_pil_image(depth): raise TypeError('img should be PIL Image. Got {}'.format(type(depth)))
        if random.random() < self.probability:
            image = np.asarray(image)
            image = Image.fromarray(image[...,list(self.indices[random.randint(0, len(self.indices) - 1)])])
        return {'image': image, 'depth': depth}

In [6]:
class ColorJitter(object):
    def __init__(self, brightness=0, contrast=0, saturation=0, hue=0):
        self.brightness = brightness
        self.contrast = contrast
        self.saturation = saturation
        self.hue = hue
        self.color_jitter = transforms.ColorJitter(brightness=self.brightness, contrast=self.contrast, saturation=self.saturation, hue=self.hue)
    def __call__(self, sample):
        image, depth = sample['image'], sample['depth']

        return {'image': self.color_jitter(image), 'depth': depth}


In [7]:
data, nyu2_train, nyu2_test  = useful.load_zip('nyu_data.zip')

Loading dataset zip file...Loaded (50688).


In [8]:
PATH_models = f'/content/gdrive/MyDrive/Diploma/Models/'
PATH_metrics = f'/content/gdrive/MyDrive/Diploma/Metrics/'

In [9]:
def load_checkpoint(model, load_path):
    state_dict = torch.load(PATH_models + load_path)
    model.load_state_dict(state_dict['model_state_dict'])
    return model

In [10]:
model_seg = smp.Unet(encoder_name="timm-regnety_004", 
                encoder_weights="imagenet",
                in_channels=3,                  
                classes=40
                )
model_seg.cuda()
model_seg = load_checkpoint(model_seg, 'seg_model_130ep.pt')


model_dep = smp.Unet(encoder_name="timm-regnety_004", 
                encoder_weights="imagenet",
                in_channels=3,                  
                classes=1
                )
model_dep.cuda()
model_dep = load_checkpoint(model_dep, 'base_timm-regnety_004_model.pth')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-regnet/regnety_004-0db870e6.pth" to /root/.cache/torch/hub/checkpoints/regnety_004-0db870e6.pth


In [11]:
model_dep.decoder.blocks[4] = model_seg.decoder.blocks[4]

In [12]:
for param in model_dep.parameters():
  param.requires_grad = True

In [13]:
for param in model_dep.decoder.blocks[4].parameters():
  param.requires_grad = False

In [20]:
model_dep = load_checkpoint(model_dep, 'transfer_model.pt')

In [15]:
batch_size = 16

In [16]:
transformed_dataset_train = DepthDataset(data, nyu2_train,transform=transforms.Compose([
                                                                                        
                                               ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4),
                                               RandomHorizontalFlip(),
                                               RandomRotate(5),
                                               ToTensorTrain()
                                                           ]))
train_loader = DataLoader(transformed_dataset_train, batch_size, shuffle=True)

In [17]:
transformed_dataset_test = DepthDataset(data, nyu2_test, transform=transforms.Compose([
                                               ToTensorTest()
                                                           ]))
test_loader = DataLoader(transformed_dataset_test, batch_size, shuffle=True)

In [21]:
def trainer(model, learning_rate = 0.0001, epochs = 10, eval_every = 1, best_test_loss = float("Inf"),
            save_path_metrics = 'transfer_metrics2.pt',
            save_path_model = 'transfer_model2.pt', optimizer=None):
  l1_criterion = nn.L1Loss()
  if optimizer is None:
    optimizer = torch.optim.Adam( model.parameters(), learning_rate)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=19008, gamma=0.1)
  global_step = 0
  running_loss = 0.0             
  test_running_loss = 0.0

  model.train()

  train_losses_list = []
  test_losses_list = []
  global_steps_list = []

  for epoch in range(epochs):

      for i, sample_batched in enumerate(train_loader):
          global_step += 1
          optimizer.zero_grad()
          image = torch.autograd.Variable(sample_batched['image'].cuda())
          depth = torch.autograd.Variable(sample_batched['depth'].cuda(non_blocking=True))
          depth_n = useful.normalize_depth(depth)
          output = model(image)

          loss_l1 = l1_criterion(output, depth_n)
          loss_ssim = torch.clamp((1 - ssim(output, depth_n, val_range = 1000.0 / 10.0)) * 0.5, 0, 1)
          loss_edges = edges_loss(output, depth_n)
          loss_depth = loss_ssim + loss_edges + loss_l1*0.1
          loss_depth.backward()
          optimizer.step()
          scheduler.step()
          running_loss += loss_depth.item()

      if global_step % eval_every == 0:

          model.eval()

          with torch.no_grad():
              for i, sample_batched in enumerate(test_loader):

                  image = torch.autograd.Variable(sample_batched['image'].cuda())
                  depth = torch.autograd.Variable(sample_batched['depth'].cuda(non_blocking=True))
                  depth_n = useful.normalize_depth(depth)
                  output = model(image)
                  loss_l1 = l1_criterion(output, depth_n)
                  loss_ssim = torch.clamp((1 - ssim(output, depth_n, val_range = 1000.0 / 10.0)) * 0.5, 0, 1)
                  loss_edges = edges_loss(output, depth_n)
                  loss_depth = loss_ssim + loss_edges + loss_l1*0.1
                  test_running_loss += loss_depth.item()
                  #TODO

                           
          average_train_loss = running_loss / len(train_loader)
          average_test_loss = test_running_loss / len(test_loader)    
          train_losses_list.append(average_train_loss)
          test_losses_list.append(average_test_loss)
          global_steps_list.append(global_step)

          running_loss = 0.0                
          test_running_loss = 0.0

          model.train()

          print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.5f}, Test Loss: {:.5f}'
                      .format(epoch+1, epochs, global_step, epochs*len(train_loader),
                              average_train_loss, average_test_loss))
          
          if (best_test_loss > average_test_loss): #and (average_train_loss < average_test_loss):

              best_test_loss = average_test_loss
              #Metrics saving
              state_dict = {'train_losses_list': train_losses_list,
                            'test_losses_list': test_losses_list,
                            'global_steps_list': global_steps_list
                           }

              torch.save(state_dict, PATH_metrics + save_path_metrics)
              print(f'Metrics saved to ==> {save_path_metrics}')
              #Model saving
              state_dict = {
                            'model_state_dict': model.state_dict(),
                            'best_test_loss': best_test_loss,
                            'optimizer_state_dict': optimizer.state_dict(),
                            'epoch': epoch
                           }
              torch.save(state_dict, PATH_models+save_path_model)
              print(f'Model saved to ==> {save_path_model}')

  print("Training finished.")

In [ ]:
trainer(model_dep, learning_rate = 0.0001, epochs = 8)

Epoch [1/8], Step [3168/25344], Train Loss: 0.60783, Test Loss: 0.12910
Metrics saved to ==> transfer_metrics.pt
Model saved to ==> transfer_model.pt
Epoch [2/8], Step [6336/25344], Train Loss: 0.36228, Test Loss: 0.12624
Metrics saved to ==> transfer_metrics.pt
Model saved to ==> transfer_model.pt
Epoch [3/8], Step [9504/25344], Train Loss: 0.26337, Test Loss: 0.12304
Metrics saved to ==> transfer_metrics.pt
Model saved to ==> transfer_model.pt
Epoch [4/8], Step [12672/25344], Train Loss: 0.22471, Test Loss: 0.11948
Metrics saved to ==> transfer_metrics.pt
Model saved to ==> transfer_model.pt
Epoch [5/8], Step [15840/25344], Train Loss: 0.21127, Test Loss: 0.11827
Metrics saved to ==> transfer_metrics.pt
Model saved to ==> transfer_model.pt
Epoch [6/8], Step [19008/25344], Train Loss: 0.20286, Test Loss: 0.11926


In [ ]:
trainer(model_dep, learning_rate = 0.001, epochs = 8)

Epoch [1/8], Step [3168/25344], Train Loss: 0.20719, Test Loss: 0.12123
Metrics saved to ==> transfer_metrics2.pt
Model saved to ==> transfer_model2.pt
Epoch [2/8], Step [6336/25344], Train Loss: 0.18058, Test Loss: 0.12214
